In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
drive='/content/gdrive/My Drive/python/fever/'  #Change the path to the correct directory where the cleaned_data.csv file is present
os.chdir(drive)
print(os.getcwd())

/content/gdrive/My Drive/python/fever


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 1.3MB 19.6MB/s 
     |████████████████████████████████| 1.1MB 67.9MB/s 
     |████████████████████████████████| 2.9MB 22.4MB/s 
     |████████████████████████████████| 890kB 56.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=9a0fb73c979ff876d2fc4e4f75a2bc5276c47a9df494bb5ce49a6e67505cdf1f
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1197470db42a44e8c4e9bbceb872ece78f5b595c7e7c1c1292e4b8752fbfb9ad
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
!pip install transformers

In [ ]:
!ls

body_index			    neural_network.py
cache_dir			    __pycache__
combined_replaced_pronouns.jsonl    runs
fever_bert_base_encased		    tfidf_vectorizer_5000_features.pickle
formatted_data_train_3_class.jsonl  title_index
license.html			    wiki-pages
__MACOSX			    wiki-pages.zip


In [ ]:
from sentence_transformers import SentenceTransformer
def get_vectors(sentences):
  model_name = 'roberta-base-nli-stsb-mean-tokens'
  model = SentenceTransformer(model_name)
  embeddings = model.encode(sentences)
  return embeddings 


In [ ]:
print(get_vectors(["This is a cat", "This is a dog"]))

100%|██████████| 461M/461M [00:08<00:00, 55.2MB/s]


[[ 0.88927007 -0.27933952  0.73015517 ... -0.36903322 -0.00148076
   1.0066966 ]
 [ 0.10002015 -0.1111249  -0.67696    ...  0.5894578   0.37469926
  -0.24015073]]


In [ ]:
import json
import sys
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import neural_network as nn
def pre_process(sentence):
    ##Replace brackets
    brackets = ['-LRB-', '-LSB-', '-RRB-', '-RSB-']
    for bracket in brackets:
        sentence = sentence.replace(bracket, " ")
    return sentence

def get_training_data():
    samples_to_use = 300000  #max=368892
    label_dict = {"SUPPORTS":0, "REFUTES": 1, "NOT ENOUGH INFO": 2}

    X_all = []
    y_all = []
    count = 0
    #train_file = "formatted_data_train.jsonl"
    train_file = "formatted_data_train_3_class.jsonl"
    fp = open(train_file, 'r')
    for line in fp:
        if count >= samples_to_use:
            break
        obj = json.loads(line.strip())
        claim = pre_process(obj['claim'])
        evidence = pre_process(obj['evidence'])
        X = (claim, evidence)
        y = label_dict[obj['label']]
        X_all.append(X)
        y_all.append(y)
        count += 1
    return X_all, y_all


In [ ]:
def train_infer_roberta():
    (X_all, y_all) = get_training_data()

    claims = [claim for (claim, _) in X_all]
    evidences = [evidence for (_, evidence) in X_all]
    print("Transforming claims")
    u = get_vectors(claims)
    print("Transforming evidences")
    v = get_vectors(evidences)
    print("Shape of u={} Shape of v={}".format(u.shape, v.shape))
    uplusv = u + v
    uminusv = abs(u - v)
    ubyv = u * v
    print("Resulting shapes=", uplusv.shape, uminusv.shape, ubyv.shape)
    all_features = np.concatenate((u, v, uplusv, uminusv, ubyv), axis=1)
    print("Shape of all=", all_features.shape)
    X_train, X_test, y_train, y_test = train_test_split(all_features, y_all, test_size=0.2, random_state=4)
    print("#Train=", len(X_train), len(y_train))
    print("#Test=", len(X_test), len(y_test))
    classifier_model_name = "baseline_classifier_infer_roberta_model.h5"
    nn.EPOCHS=20
    nn.fit_predict(X_train, y_train, X_test, y_test, classifier_model_name)

train_infer_roberta()

Transforming claims
Transforming evidences
Shape of u=(300000, 768) Shape of v=(300000, 768)
Resulting shapes= (300000, 768) (300000, 768) (300000, 768)
Shape of all= (300000, 3840)
#Train= 240000 240000
#Test= 60000 60000
Epoch 1/20
235/235 [==============================] - 2s 9ms/step - loss: 0.3127 - accuracy: 0.9483
Epoch 2/20
235/235 [==============================] - 2s 9ms/step - loss: 0.1782 - accuracy: 0.9585
Epoch 3/20
235/235 [==============================] - 2s 9ms/step - loss: 0.1639 - accuracy: 0.9623
Epoch 4/20
235/235 [==============================] - 2s 9ms/step - loss: 0.1543 - accuracy: 0.9651
Epoch 5/20
235/235 [==============================] - 2s 9ms/step - loss: 0.1452 - accuracy: 0.9675
Epoch 6/20
235/235 [==============================] - 2s 9ms/step - loss: 0.1384 - accuracy: 0.9693
Epoch 7/20
235/235 [==============================] - 2s 9ms/step - loss: 0.1335 - accuracy: 0.9707
Epoch 8/20
235/235 [==============================] - 2s 9ms/step - loss: 0.1